# Data Analysis of Scraped Tweets

The JSON data is first read and useful attributes are extracted. This information can be used to extract sentiments around the keyword.

In [1]:
import json
import pandas as pd
from twitter_trends import TwitterBot

In [2]:
bot = TwitterBot()
bot.fetch_data(['covid', 'corona'], max_tweets = 1000) #Scraped data is stored at data/tweets.json

Getting tweet #1
Getting tweet #2
Getting tweet #3
Getting tweet #4
Getting tweet #5
Getting tweet #6
Getting tweet #7
Getting tweet #8
Getting tweet #9
Getting tweet #10
Getting tweet #11
Getting tweet #12
Getting tweet #13
Getting tweet #14
Getting tweet #15
Getting tweet #16
Getting tweet #17
Getting tweet #18
Getting tweet #19
Getting tweet #20
Getting tweet #21
Getting tweet #22
Getting tweet #23
Getting tweet #24
Getting tweet #25
Getting tweet #26
Getting tweet #27
Getting tweet #28
Getting tweet #29
Getting tweet #30
Getting tweet #31
Getting tweet #32
Getting tweet #33
Getting tweet #34
Getting tweet #35
Getting tweet #36
Getting tweet #37
Getting tweet #38
Getting tweet #39
Getting tweet #40
Getting tweet #41
Getting tweet #42
Getting tweet #43
Getting tweet #44
Getting tweet #45
Getting tweet #46
Getting tweet #47
Getting tweet #48
Getting tweet #49
Getting tweet #50
Getting tweet #51
Getting tweet #52
Getting tweet #53
Getting tweet #54
Getting tweet #55
Getting tweet #56
G

Getting tweet #441
Getting tweet #442
Getting tweet #443
Getting tweet #444
Getting tweet #445
Getting tweet #446
Getting tweet #447
Getting tweet #448
Getting tweet #449
Getting tweet #450
Getting tweet #451
Getting tweet #452
Getting tweet #453
Getting tweet #454
Getting tweet #455
Getting tweet #456
Getting tweet #457
Getting tweet #458
Getting tweet #459
Getting tweet #460
Getting tweet #461
Getting tweet #462
Getting tweet #463
Getting tweet #464
Getting tweet #465
Getting tweet #466
Getting tweet #467
Getting tweet #468
Getting tweet #469
Getting tweet #470
Getting tweet #471
Getting tweet #472
Getting tweet #473
Getting tweet #474
Getting tweet #475
Getting tweet #476
Getting tweet #477
Getting tweet #478
Getting tweet #479
Getting tweet #480
Getting tweet #481
Getting tweet #482
Getting tweet #483
Getting tweet #484
Getting tweet #485
Getting tweet #486
Getting tweet #487
Getting tweet #488
Getting tweet #489
Getting tweet #490
Getting tweet #491
Getting tweet #492
Getting twee

Getting tweet #880
Getting tweet #881
Getting tweet #882
Getting tweet #883
Getting tweet #884
Getting tweet #885
Getting tweet #886
Getting tweet #887
Getting tweet #888
Getting tweet #889
Getting tweet #890
Getting tweet #891
Getting tweet #892
Getting tweet #893
Getting tweet #894
Getting tweet #895
Getting tweet #896
Getting tweet #897
Getting tweet #898
Getting tweet #899
Getting tweet #900
Getting tweet #901
Getting tweet #902
Getting tweet #903
Getting tweet #904
Getting tweet #905
Getting tweet #906
Getting tweet #907
Getting tweet #908
Getting tweet #909
Getting tweet #910
Getting tweet #911
Getting tweet #912
Getting tweet #913
Getting tweet #914
Getting tweet #915
Getting tweet #916
Getting tweet #917
Getting tweet #918
Getting tweet #919
Getting tweet #920
Getting tweet #921
Getting tweet #922
Getting tweet #923
Getting tweet #924
Getting tweet #925
Getting tweet #926
Getting tweet #927
Getting tweet #928
Getting tweet #929
Getting tweet #930
Getting tweet #931
Getting twee

In [3]:
with open('data/tweets.json') as f:
    d = f.read()
    tweets = json.loads(d) #tweets is a list of dictionary

The possible attributes in the tweets are:

['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
'in_reply_to_status_id', 'in_reply_to_status_id_str',
'in_reply_to_user_id', 'in_reply_to_user_id_str',
'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
'contributors', 'retweeted_status', 'quoted_status_id',
'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink',
'is_quote_status', 'quote_count', 'reply_count', 'retweet_count',
'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level',
'lang', 'timestamp_ms', 'possibly_sensitive', 'extended_entities',
'display_text_range', 'extended_tweet']

Note that not all tweets would have all of these attributes present. Some attributes such as extended_tweet may be present only in fraction of tweets.

In [4]:
processed = []
for tweet in tweets:
    if tweet['lang'] != 'en':
        continue
    t = {}
    if 'extended_tweet' in tweet.keys():
        t['text'] = tweet['extended_tweet']['full_text']
    else:
        t['text'] = tweet['text']
    
    processed.append(t)

In [5]:
df = pd.DataFrame(processed)

In [6]:
df.head()

,text
0,"RT @patton6966: First impeachment, then Covid-..."
1,"RT @patton6966: First impeachment, then Covid-..."
2,RT @DavidDTSS: in 2-3 weeks the media is going...
3,RT @i_Shoaibbinadam: And today Pakistan crosse...
4,RT @malissaali: I am confident we can win agai...


# Sentiment using NLTK

Using the VADER SentimentIntensityAnalyzer

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [8]:
sentiment_analyser = SentimentIntensityAnalyzer()

In [9]:
sentences = list(df['text']) #Get the list of tweets

After Vader Analysis:

neg: [0, 1] <br />
neu: [0, 1] <br />
pos: [0, 1] <br />
compound: [-1, 1] <br />
<br />
If compound >= 0.05, label = +1 (Positive) <br />
If compound <= -0.05, label = -1 (Negative) <br />
Otherwise, label = 0 (Neutral)

In [10]:
data = []
for sentence in sentences:
    row = {'text': sentence}
    sentiment = sentiment_analyser.polarity_scores(sentence)
    sentiment['compound'] = sentiment['compound']
    if sentiment['compound'] >= 0.05:
        sentiment['label'] = 1
    elif sentiment['compound'] <= -0.05:
        sentiment['label'] = -1
    else:
        sentiment['label'] = 0
        
    row.update(sentiment)
    data.append(row)

In [11]:
df_final = pd.DataFrame(data)

In [12]:
df_final.head()

,text,neg,neu,pos,compound,label
0,"RT @patton6966: First impeachment, then Covid-...",0.247,0.753,0.000,-0.7840,-1
1,"RT @patton6966: First impeachment, then Covid-...",0.247,0.753,0.000,-0.7840,-1
2,RT @DavidDTSS: in 2-3 weeks the media is going...,0.000,0.925,0.075,0.1779,1
3,RT @i_Shoaibbinadam: And today Pakistan crosse...,0.000,1.000,0.000,0.0000,0
4,RT @malissaali: I am confident we can win agai...,0.000,0.650,0.350,0.7906,1


In [13]:
df_final.describe()

,neg,neu,pos,compound,label
count,470.000000,470.000000,470.000000,470.000000,470.000000
mean,0.110338,0.813191,0.076483,-0.099568,-0.117021
std,0.129948,0.152799,0.098964,0.492231,0.862094
min,0.000000,0.158000,0.000000,-0.987200,-1.000000
25%,0.000000,0.718000,0.000000,-0.542300,-1.000000
50%,0.078000,0.838000,0.014000,0.000000,0.000000
75%,0.198500,0.925000,0.132000,0.273200,1.000000
max,0.611000,1.000000,0.565000,0.904800,1.000000


In [14]:
df_final['label'].describe()

count    470.000000
mean      -0.117021
std        0.862094
min       -1.000000
25%       -1.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: label, dtype: float64